In [47]:
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf

In [48]:
PATH = os.path.join(os.path.dirname('../TrashClassification/dataset-resized/'))

train_dir = os.path.join(PATH, 'train')
validation_dir = os.path.join(PATH, 'validation')

BATCH_SIZE = 8
IMG_SIZE = (512, 384)

train_dataset = tf.keras.utils.image_dataset_from_directory(train_dir,
                                                            shuffle=True,
                                                            batch_size=BATCH_SIZE,
                                                            image_size=IMG_SIZE)


Found 2197 files belonging to 6 classes.


In [49]:
validation_dataset = tf.keras.utils.image_dataset_from_directory(validation_dir,
                                                                 shuffle=True,
                                                                 batch_size=BATCH_SIZE,
                                                                 image_size=IMG_SIZE)


Found 330 files belonging to 6 classes.


In [50]:
val_batches = tf.data.experimental.cardinality(validation_dataset)
test_dataset = validation_dataset.take(val_batches // 5)
validation_dataset = validation_dataset.skip(val_batches // 5)


In [51]:
print('Number of validation batches: %d' % tf.data.experimental.cardinality(validation_dataset))
print('Number of test batches: %d' % tf.data.experimental.cardinality(test_dataset))


Number of validation batches: 34
Number of test batches: 8


In [52]:
AUTOTUNE = tf.data.AUTOTUNE

train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
validation_dataset = validation_dataset.prefetch(buffer_size=AUTOTUNE)
test_dataset = test_dataset.prefetch(buffer_size=AUTOTUNE)


In [53]:
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.RandomFlip('horizontal'),
  tf.keras.layers.RandomRotation(0.2),
])


In [54]:
preprocess_input = tf.keras.applications.resnet50.preprocess_input

# Create the base model from the pre-trained model MobileNet V2
IMG_SHAPE = IMG_SIZE + (3,)
base_model = tf.keras.applications.ResNet50(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

image_batch, label_batch = next(iter(train_dataset))
feature_batch = base_model(image_batch)
print(feature_batch.shape)



(8, 16, 12, 2048)


In [55]:
base_model.trainable = False
# Let's take a look at the base model architecture
base_model.summary()


Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_6 (InputLayer)           [(None, 512, 384, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 518, 390, 3)  0           ['input_6[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 256, 192, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                           

In [56]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
feature_batch_average = global_average_layer(feature_batch)
print(feature_batch_average.shape)


(8, 2048)


In [57]:
prediction_layer = tf.keras.layers.Dense(1)
prediction_batch = prediction_layer(feature_batch_average)
print(prediction_batch.shape)


(8, 1)


In [58]:
inputs = tf.keras.Input(shape=(512, 384, 3))
x = data_augmentation(inputs)
x = preprocess_input(x)
x = base_model(x, training=False)
x = global_average_layer(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = prediction_layer(x)
model = tf.keras.Model(inputs, outputs)


In [59]:
base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])


In [60]:
model.summary()


Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 512, 384, 3)]     0         
                                                                 
 sequential_2 (Sequential)   (None, 512, 384, 3)       0         
                                                                 
 tf.__operators__.getitem_3   (None, 512, 384, 3)      0         
 (SlicingOpLambda)                                               
                                                                 
 tf.nn.bias_add_3 (TFOpLambd  (None, 512, 384, 3)      0         
 a)                                                              
                                                                 
 resnet50 (Functional)       (None, 16, 12, 2048)      23587712  
                                                                 
 global_average_pooling2d_2   (None, 2048)             0   

In [61]:
len(model.trainable_variables)


2

In [62]:
initial_epochs = 10

loss0, accuracy0 = model.evaluate(validation_dataset)


34/34 [==============================] - 72s 2s/step - loss: 0.0301 - accuracy: 0.2782


In [63]:
print("initial loss: {:.2f}".format(loss0))
print("initial accuracy: {:.2f}".format(accuracy0))


initial loss: 0.03
initial accuracy: 0.28


In [64]:
history = model.fit(train_dataset,
                    epochs=initial_epochs,
                    validation_data=validation_dataset)


Epoch 1/10
275/275 [==============================] - 695s 3s/step - loss: -9.8011 - accuracy: 0.2039 - val_loss: -15.0046 - val_accuracy: 0.2331
Epoch 2/10
275/275 [==============================] - 671s 2s/step - loss: -28.7848 - accuracy: 0.1939 - val_loss: -29.4872 - val_accuracy: 0.2143
Epoch 3/10
275/275 [==============================] - 658s 2s/step - loss: -47.7000 - accuracy: 0.1939 - val_loss: -43.1873 - val_accuracy: 0.2218
Epoch 4/10
275/275 [==============================] - 659s 2s/step - loss: -67.1035 - accuracy: 0.1939 - val_loss: -56.1582 - val_accuracy: 0.2368
Epoch 5/10
275/275 [==============================] - 687s 3s/step - loss: -86.3936 - accuracy: 0.1939 - val_loss: -69.1968 - val_accuracy: 0.2218
Epoch 6/10
275/275 [==============================] - 669s 2s/step - loss: -105.6006 - accuracy: 0.1939 - val_loss: -90.3479 - val_accuracy: 0.2068
Epoch 7/10
 93/275 [=========>....................] - ETA: 6:36 - loss: -112.4652 - accuracy: 0.2030

KeyboardInterrupt: 

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()
